In [1]:
# %% [markdown]
# # 🏥 HealthGuardian AI - Colab Version
# Intelligent Medical Triage System

# %%
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import os
from IPython.display import display, Markdown
import ipywidgets as widgets

# Load or create training data
def load_data():
    sample_data = {
        'symptoms': [
            'fever headache runny_nose', 'high_fever chills body_ache',
            'chest_pain shortness_breath', 'abdominal_pain nausea vomiting',
            'dry_cough shortness_breath', 'mild_headache', 'skin_rashes fever',
            'sore_throat cough', 'dizziness blurred_vision', 'ear_pain fever'
        ],
        'condition': [
            'flu', 'flu', 'cardiac_emergency', 'gastroenteritis',
            'covid_suspicion', 'headache', 'dengue_suspicion', 'tonsillitis',
            'blood_pressure', 'otitis'
        ],
        'urgency': ['low', 'medium', 'high', 'medium', 'high',
                   'low', 'high', 'medium', 'high', 'medium'],
        'recommendations': [
            'rest hydration monitor_temperature', 'rest hydration fever_medication consult_24h',
            'seek_immediate_care', 'light_diet hydration consult_48h',
            'isolation covid_test urgent_consultation', 'rest pain_reliever',
            'rest hydration seek_urgent_care', 'gargle pain_reliever medical_consultation',
            'check_blood_pressure seek_urgent_care', 'pain_reliever medical_consultation'
        ]
    }
    return pd.DataFrame(sample_data)

# Train model
def train_model(df):
    vectorizer = TfidfVectorizer(max_features=100)
    X = vectorizer.fit_transform(df['symptoms'])
    y_condition = df['condition']
    y_urgency = df['urgency']

    model_condition = RandomForestClassifier(n_estimators=50, random_state=42)
    model_urgency = RandomForestClassifier(n_estimators=50, random_state=42)

    model_condition.fit(X, y_condition)
    model_urgency.fit(X, y_urgency)

    return vectorizer, model_condition, model_urgency

# Load data and train model
df = load_data()
vectorizer, model_condition, model_urgency = train_model(df)

# %%
# Interactive interface
display(Markdown("## 📋 Describe Your Symptoms"))

symptoms_input = widgets.Textarea(
    value='',
    placeholder='Example: fever, headache, runny nose, nausea...',
    description='Symptoms:',
    layout=widgets.Layout(width='80%', height='100px')
)

analyze_button = widgets.Button(description="🔍 Analyze Symptoms", button_style='primary')

output = widgets.Output()

def on_analyze_click(b):
    with output:
        output.clear_output()
        symptoms_text = symptoms_input.value.strip()

        if symptoms_text:
            # Preprocess and predict
            symptoms_vec = vectorizer.transform([symptoms_text])

            condition_pred = model_condition.predict(symptoms_vec)[0]
            urgency_pred = model_urgency.predict(symptoms_vec)[0]

            # Display results
            display(Markdown("---"))
            display(Markdown("## 📊 Triage Results"))

            # Urgency level
            urgency_colors = {'low': '🟢', 'medium': '🟡', 'high': '🔴'}
            urgency_descs = {'low': 'LOW - Consultation in 48-72h',
                           'medium': 'MEDIUM - Consultation in 24-48h',
                           'high': 'HIGH - Seek immediate care'}

            display(Markdown(f"### {urgency_colors[urgency_pred]} URGENCY LEVEL: {urgency_descs[urgency_pred]}"))
            display(Markdown(f"### 🩺 POSSIBLE CONDITIONS: {condition_pred.replace('_', ' ').title()}"))

            # Warning
            display(Markdown("---"))
            display(Markdown("""
            ⚠️ **IMPORTANT NOTICE:**
            This is a preliminary triage system and **DOES NOT REPLACE**
            evaluation by a healthcare professional.
            Always consult a doctor for accurate diagnosis.
            """))
        else:
            display(Markdown("❌ **Please describe your symptoms for analysis.**"))

analyze_button.on_click(on_analyze_click)

display(symptoms_input)
display(analyze_button)
display(output)

## 📋 Describe Your Symptoms

Textarea(value='', description='Symptoms:', layout=Layout(height='100px', width='80%'), placeholder='Example: …

Button(button_style='primary', description='🔍 Analyze Symptoms', style=ButtonStyle())

Output()